In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torchvision.datasets import CocoDetection

# Backbone Init

In [76]:
class ResNetBackBone(nn.Module):
    def __init__(self, pretrained_model = None):
        if pretrained_model is None:
            pretrained_model=torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        super(ResNetBackBone, self).__init__()
        self.features = nn.Sequential(
            *list(pretrained_model.children())[:-2]
        )

    def freeze_weights(self):
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.features(x)
        return x

In [62]:
resnet18 = ResNetBackBone(pretrained_model=torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True))

Using cache found in /Users/bartoszgawin/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [63]:
resnet18.freeze_weights()

# Building the Transformer blocks

In [70]:
class EncoderBlock(nn.Module):
    def __init__(self, model_dim:int, nheads:int, dim_feedforward:int = 248, dropout:int = 0):
        super(EncoderBlock, self).__init__()
        self.attention_matrices = None

        self.multi_head_attention = nn.MultiheadAttention(embed_dim=model_dim, num_heads=nheads, batch_first=True)
        self.norm_1 = nn.LayerNorm(model_dim)
        self.dropout_1 = nn.Dropout(p = dropout)

        self.fully_connected = nn.Sequential(
            nn.Linear(model_dim, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, model_dim)
        )

        self.dropout_2 = nn.Dropout(p = dropout)
        self.norm_2 = nn.LayerNorm(model_dim)

    def forward(self, X, positional_encoding):
        q = k = X + positional_encoding
        attention_output, self.attention_matrices = self.multi_head_attention(query = q, key = k, value = X, average_attn_weights=True)
        attention_output = self.norm_1(X + self.dropout_1(attention_output))
        fc_output = self.fully_connected(attention_output)
        output = self.norm_2(attention_output + self.dropout_2(fc_output))
        return output

In [167]:
class DecoderBlock(nn.Module):
    def __init__(self, model_dim:int , nheads:int, dim_feedforward:int = 248, dropout:int = 0):
        super(DecoderBlock, self).__init__()
        self.attention_matrices_1 = None
        self.attention_matrices_2 = None

        self.multi_head_attention_1 = nn.MultiheadAttention(embed_dim=model_dim, num_heads=nheads, batch_first=True)
        self.norm_1 = nn.LayerNorm(model_dim)
        self.dropout_1 = nn.Dropout(p = dropout)


        self.multi_head_attention_2 = nn.MultiheadAttention(embed_dim=model_dim, num_heads=nheads, batch_first=True)
        self.norm_2 = nn.LayerNorm(model_dim)
        self.dropout_2 = nn.Dropout(p = dropout)

        self.fully_connected = nn.Sequential(
            nn.Linear(model_dim, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, model_dim)
        )

        self.norm_3 = nn.LayerNorm(model_dim)
        self.dropout_3 = nn.Dropout(p = dropout)

    def forward(self, object_query, memory, positional_encoding):
        batch_size = memory.shape[:-2]
        q = k = object_query + positional_encoding
        attention_1_output, self.attention_matrices_1 = self.multi_head_attention_1(query=q, key=k, value=object_query, average_attn_weights=True)
        attention_1_output = self.norm_1(object_query + self.dropout_1(attention_1_output))
        mem_k = memory + positional_encoding
        mem_q = attention_1_output + positional_encoding
        if object_query.shape[0] != batch_size[0]:
            mem_q = mem_q.repeat(*batch_size, 1, 1)
        attention_2_output, self.attention_matrices_2 = self.multi_head_attention_2(query=mem_q, key=mem_k, value=memory, average_attn_weights=True)
        attention_2_output = self.norm_2(attention_1_output + self.dropout_2(attention_2_output))
        fc_output = self.fully_connected(attention_2_output)
        output = self.norm_3(attention_2_output + self.dropout_3(fc_output))
        return output

In [168]:
class DETR(nn.Module):
    def __init__(self, input_size:tuple[int, int], num_classes:int, model_dim:int=128, nheads:int=8, num_encoder_layers:int=1,
               num_decoder_layers:int=1, dim_feedforward:int=512, dropout:int=0.1):
        super(DETR, self).__init__()

        self.backbone = ResNetBackBone()
        self.backbone.freeze_weights()

        trace_backbone_dims = torch.rand(2, 3, *input_size)
        _, backbone_channels, width, height = self.backbone(trace_backbone_dims).shape
        self.conv = nn.Conv2d(backbone_channels, model_dim, 1)

        # Paper isn't fully clear about how they handle pictures of different dimensions
        # For the sake of sanity I assume a fixed size picture input like in most models
        self.query_pos = nn.Parameter(torch.rand(int(width*height), model_dim))
        self.pos_encode = nn.Parameter(torch.rand(int(width*height), model_dim))

        self.encoder = nn.ModuleList()
        for _ in range(num_encoder_layers):
            self.encoder.append(EncoderBlock(model_dim,nheads,dim_feedforward,dropout))

        self.decoder = nn.ModuleList()
        for _ in range(num_decoder_layers):
            self.decoder.append(DecoderBlock(model_dim,nheads,dim_feedforward,dropout))

        self.linear_class = nn.Linear(model_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(model_dim, 4)


    def forward(self, X):
        X = self.backbone(X)
        X = self.conv(X)
        mem = X.flatten(2).permute(0, 2, 1)
        for block in self.encoder:
            mem = block(mem, self.pos_encode)
        h = self.decoder[0](self.query_pos.unsqueeze(0), mem, self.pos_encode)
        for block in self.decoder[1:]:
            h = block(h, mem, self.pos_encode)
        return self.linear_class(h), self.linear_bbox(h)

# Checking the Code

In [169]:
batch_size = 10
h = 224
w = 224
num_classes = 10

detr = DETR((h, w), num_classes, num_encoder_layers = 2, num_decoder_layers = 2)
input = torch.randn(batch_size, 3, h, w)
cls, bbox = detr(input)
print(cls.shape)
print(bbox.shape)

Using cache found in /Users/bartoszgawin/.cache/torch/hub/pytorch_vision_v0.10.0


torch.Size([10, 49, 11])
torch.Size([10, 49, 4])


In [102]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

decoder_block = DecoderBlock(D, H)
input = torch.randn(batch_size, N+1, D).permute(2, 0, 1)
print(input.shape)
output = decoder_block(input, input, input)
print(input.shape)

torch.Size([64, 10, 50])


AssertionError: was expecting embedding dimension of 64, but got 50

In [170]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

decoder_block = DecoderBlock(D, H)
obj = torch.randn(batch_size, 100, D)
mem = torch.randn(batch_size, N+1, D)
pos_enc = torch.randn(1, 100, D)
output = decoder_block(obj, mem, pos_enc)
print(output.shape)

RuntimeError: The size of tensor a (50) must match the size of tensor b (100) at non-singleton dimension 1

In [75]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

encoder_block = EncoderBlock(D, H)
input = torch.randn(batch_size, N+1, D)
pos_enc = torch.randn(1, N+1, D)
print(input.shape)
output = encoder_block(input, pos_enc)
print(output.shape)

torch.Size([10, 50, 64])
torch.Size([10, 50, 64])


In [23]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
dimensions = int(D / H)

multi_head_attention = nn.MultiheadAttention(D, H, batch_first=True)
input = torch.randn(batch_size, N + 1, D)
output = multi_head_attention(input, input, input, average_attn_weights=True)
print(output[0].shape)

torch.Size([10, 50, 64])


In [11]:
input.permute(0, 2, 1).permute(0, 2, 1).shape

torch.Size([10, 50, 64])

In [10]:
batch_size = 10
patch_size = 4
height_and_width = 28
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

multi_head_attention = EncoderBlock(D, H)
input = torch.randn(batch_size, N+1, D)
print(input.shape)
output = multi_head_attention(input, input)

torch.Size([10, 50, 64])
attention_output dim: torch.Size([10, 50, 64])


In [88]:
batch_size = 10
patch_size = 4
h = 50
w = 50
D = 64
N = int(height_and_width**2 / patch_size**2)
H = 8
C = 64
dimensions = int(D / H)

row_embed = torch.rand(50, D // 2)
col_embed = torch.rand(50, D // 2)

print(col_embed[:w].unsqueeze(0).repeat(h, 1, 1).shape)
print(row_embed[:h].unsqueeze(1).repeat(1, w, 1).shape)

pos = torch.cat([
    col_embed[:w].unsqueeze(0).repeat(h, 1, 1),
    row_embed[:h].unsqueeze(1).repeat(1, w, 1),
    ], dim=-1).flatten(0, 1).unsqueeze(1)

pos.shape

torch.Size([51, 50, 32])
torch.Size([50, 51, 32])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 51 but got size 50 for tensor number 1 in the list.

In [50]:
from torchvision import transforms
from medmnist import DermaMNIST
from torch.utils.data import DataLoader

train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

train_download = DermaMNIST(split='train', transform=train_transformations, download=True)

Using downloaded and verified file: /Users/bartoszgawin/.medmnist/dermamnist.npz


In [97]:
w = 224
h = 224
D = 512
x = torch.rand(2, 3, w, h)
# print(x.shape)

row_embed = torch.rand(50, D // 2)
col_embed = torch.rand(50, D // 2)

output = resnet18(x)
print(output.shape)
print(output.flatten(2).shape)
print(output.flatten(2).permute(2, 0, 1).shape)

row_embed = torch.rand(480, D // 2)
col_embed = torch.rand(480, D // 2)

# print(col_embed[:w].unsqueeze(0).repeat(h, 1, 1).shape)
# print(row_embed[:h].unsqueeze(1).repeat(1, w, 1).shape)

pos = torch.cat([
    col_embed[:w].unsqueeze(0).repeat(h, 1, 1),
    row_embed[:h].unsqueeze(1).repeat(1, w, 1),
], dim=-1).flatten(0, 1).unsqueeze(1)

print(f'pos shape: {pos.shape}')
print((pos + output.flatten(2).permute(2,0,1))[0].shape)

torch.Size([2, 512, 7, 7])
torch.Size([2, 512, 49])
torch.Size([49, 2, 512])
pos shape: torch.Size([50176, 1, 512])


RuntimeError: The size of tensor a (50176) must match the size of tensor b (49) at non-singleton dimension 0

In [59]:
resnet18

ResNetBackBone(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

# Source Code of ResNet

In [16]:
# Source code of ResNet
class ResNet(nn.Module):
    def __init__(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            layers: List[int],
            num_classes: int = 1000,
            zero_init_residual: bool = False,
            groups: int = 1,
            width_per_group: int = 64,
            replace_stride_with_dilation: Optional[List[bool]] = None,
            norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
            self,
            block: Type[Union[BasicBlock, Bottleneck]],
            planes: int,
            blocks: int,
            stride: int = 1,
            dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

NameError: name 'Type' is not defined